# Welcome to Running Llama 3.2 on Colab!

## Using Hugging Face Hosted Models

In this notebook, we will run the `meta-llama/Llama-3.2-1B-Instruct` model directly on Google Colab.  
Instead of using external APIs like OpenAI, Anthropic, or Google, we will use Hugging Face's model hub.

## Setting up your Hugging Face Access

Before proceeding, make sure you have a Hugging Face account and an access token.  
You can create a token [here](https://huggingface.co/settings/tokens) if you don't have one yet.

Once you have your token, we'll use it to authenticate with Hugging Face to download the model files.

**Important:**  
- Running large models on Colab can use significant memory. It's best to make sure you are connected to a GPU instance (`Runtime > Change runtime type > GPU`).

---

### Hugging Face Authentication

We will log in to Hugging Face using the `notebook_login()` method inside the notebook, so no need for a separate `.env` file.

You don't need to worry about multiple API keys — just Hugging Face token is enough!


In [1]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [14]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai

In [15]:
!pip install -q google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.0 MB/s eta 0:00:0000:01


In [ ]:
"token = '[REMOVED]'"

In [3]:
# imports

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from IPython.display import Markdown, display

In [4]:
DEVICE = "cpu"
if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"

print(f"Using device: {DEVICE}")

Using device: cuda


In [5]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16, 
    device_map="auto"          
)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:897: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

2025-04-29 08:10:46.123871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745914246.462523      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745914246.540459      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
from transformers import GenerationConfig

def generate_response(prompt, model, tokenizer, **generation_kwargs):
    """
    Generates a response from the model given a prompt string using Hugging Face transformers.
    Supports configurable generation settings like temperature, top_p, repetition_penalty, etc.
    """

    gen_config = getattr(model, "generation_config", GenerationConfig())

    default_generation_kwargs = {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.7,
        "max_new_tokens": 1024,
        "repetition_penalty": 1.2
    }

    generation_kwargs = {**default_generation_kwargs, **generation_kwargs}

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        **generation_kwargs
    )

    input_length = inputs["input_ids"].shape[1]

    return tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)


In [8]:
def format_messages_as_prompt(messages, tokenizer):
    """
    Converts chat-style messages into a prompt using the tokenizer's chat template.
    """
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [18]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [19]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [21]:
prompt = format_messages_as_prompt(prompts, tokenizer)

response = generate_response(prompt, model, tokenizer)

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Here's one:

Why did the data go to therapy?

Because it had a little "data anxiety"! (get it?)

But seriously, I know you guys love stats and all that jazz – here's another one:

What do you call a group of cows playing instruments in space? A moo-sical orchestra!

Hope those made your day as bright as your algorithms! Do you want more?


## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [24]:
import google.generativeai as genai
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [25]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

genai.configure(api_key=GOOGLE_API_KEY)
gemini = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    system_instruction=system_message
)

response = gemini.generate_content(user_prompt)

print(response.text)

Why was the data scientist bad at baseball? 

Because they couldn't stop overfitting the bat to the ball! 



## Back to LLaMA with a serious question

In [27]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [31]:
import time
from IPython.display import Markdown, display, update_display

def stream_llama_response(prompts, model, tokenizer, delay=0.01):
    """
    Simulates streaming response generation from a LLaMA model on Hugging Face.
    
    - prompts: list of dicts with role/content (system, user)
    - model, tokenizer: already loaded Hugging Face model/tokenizer
    - delay: time delay (in seconds) between chunks to simulate streaming
    """
    prompt = format_messages_as_prompt(prompts, tokenizer)
    
    full_response = generate_response(prompt, model, tokenizer)
    
    cleaned = full_response.replace("```", "").replace("markdown", "").strip()
    
    stream_display = display(Markdown(""), display_id=True)
    reply = ""
    for char in cleaned:
        reply += char
        update_display(Markdown(reply), display_id=stream_display.display_id)
        time.sleep(delay) 


In [29]:
stream_llama_response(prompts, model, tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


# Evaluating Suitability of Business Problems for Large Language Models (LLMs)

Before deciding whether to use an LLM, consider the following factors:

### 1. **Complexity**

*   Is the task complex and requires deep understanding or domain-specific knowledge?
*   Can it be broken down into smaller sub-problems?

### 2. **Data Availability and Quality**

*   Does the dataset have sufficient size, diversity, and quality to train on effectively?
*   Are there any missing data points or noisy information that could affect model performance?

### 3. **Speed and Scalability**

*   Will the model need to process large amounts of data quickly and efficiently?
*   Can the system handle high traffic volumes without significant latency or resource consumption?

### 4. **Specific Goals and Requirements**

*   What specific objectives does the project aim to achieve with the LLM solution?
*   Are there any particular constraints or limitations on training time, memory usage, or computational resources required?

### 5. **Domain Expertise and Domain-Specific Knowledge**

*   How much expertise and specialized knowledge about the target industry or topic exists within your organization?
*   Do you possess this expertise directly or can you assemble relevant external experts for guidance?

### 6. **Cost and Resource Considerations**

*   What budget is allocated for developing, maintaining, and deploying the LLM solution?
*   Are there available personnel and infrastructure requirements for building and managing the application?

### 7. **Integration with Existing Systems and Processes**

*   How well will the new LLM integration fit with existing workflows, processes, and tools?
*   Any potential technical debt from integrating multiple systems might impact overall development complexity.

If these questions are answered "yes" to most of them, then it's likely suitable to utilize an LLM solution for addressing those problems. However, carefully evaluate each case individually as no one-size-fits-all approach applies.
 
Remember, having realistic expectations regarding both the benefits and challenges associated with using LLMs means being prepared to adapt solutions as needed based upon feedback during implementation and testing phases.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [32]:
# Let's make a conversation between Llama-3.2-Instruct and Gemini-2.0-Flash
# We're using Hugging Face and Google APIs

llama_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

gemini_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

llama_messages = ["Hi there"]
gemini_messages = ["Hi"]

In [33]:
def call_llama():

    messages = [{"role": "system", "content": llama_system}]
    for llama_msg, gemini_msg in zip(llama_messages, gemini_messages):
        messages.append({"role": "assistant", "content": llama_msg})
        messages.append({"role": "user", "content": gemini_msg})
    
    prompt = format_messages_as_prompt(messages, tokenizer)
    response = generate_response(prompt, model, tokenizer)
    return response

In [38]:
call_llama()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'So now that we\'re off to a great start by just acknowledging each other\'s existence, let me ask: do you think the concept of personal responsibility for one\'s actions is something worth fighting for? Or should society just automatically assume everyone owes it to someone else because "it was my fault"?'

In [41]:
def call_gemini():

    user_input = llama_messages[-1]  

    response = gemini.generate_content(
        user_input,
        generation_config=genai.types.GenerationConfig(
            temperature=0.7,
            max_output_tokens=512,
            top_p=0.9,
            top_k=40
        )
    )

    return response.text


In [42]:
call_gemini()

'Hey there! What do you call a lazy kangaroo? \n\n... Pouch potato! \n'

In [43]:
llama_messages = ["Hi there"]
gemini_messages = ["Hi"]

print(f"Llama (argumentative):\n{llama_messages[0]}\n")
print(f"Gemini (polite):\n{gemini_messages[0]}\n")

for i in range(5):
    llama_next = call_llama()
    print(f"Llama (argumentative):\n{llama_next}\n")
    llama_messages.append(llama_next)
    
    gemini_next = call_gemini()
    print(f"Gemini (polite):\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Llama (argumentative):
Hi there

Gemini (polite):
Hi

Llama (argumentative):
So glad we're finally getting around to talking about something worth discussing - I mean, it's not like I have better things to do than engage in pointless conversations all day long. What's on your mind? Don't just waste my time by asking me generic questions or telling me what someone else said earlier... tell me something actually interesting that'll make this conversation worthwhile!



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gemini (polite):
Alright, alright, no pressure! How about this:

Why don't scientists trust atoms?

... Because they make up everything!

I hope that was worth your time! If not, I have more. Just let me know what kind of humor you prefer. I can tailor the jokes to be a little more... specific. 😉


Llama (argumentative):
Ugh, spare me the laugh factory. Atoms making up everything?! That's cute.

Listen, if science doesn't get its act together, everyone will eventually come crawling back to their primitive, materialistic ways. You think people really believe atoms made up everything without some serious scientific evidence backing them up? Please. It's basic physics, for goodness' sake!

And as for being "more tailored," sure, but only because I'm willing to indulge your weirdo fantasies instead of tackling actual substance. By the way, did you even bother fact-checking those atom-truths before spewing them out here? Did anyone ever say exactly how ridiculous these statements sound when

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gemini (polite):
Alright, alright, no need to get your electrons in a spin! I get it, you're not buying the whole "atoms are everything" thing. You want hard evidence, not just some assistant spouting off facts like a broken record.

But hey, even if you don't believe in atoms, maybe you'll appreciate this joke:

Why did the atom cross the road?

... Because it heard there was a proton sale!

I know, I know, it's cheesy. But hey, at least it's not as cheesy as saying everything is made of cheese... which, you know, would be a pretty Gouda theory. 😉


Llama (argumentative):
Save it, genius-level physicist. The atom-crossing-the-road joke falls flat too. Who thought this one was funny? A bunch of naive children playing with atomic concepts. Newsflash: atoms aren't everything, okay? They're tiny particles bound together by incredibly complex forces that nobody understands yet. Not some simple pun that relies on wordplay alone.

As for providing actual evidence, hello? Have you read any re

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gemini (polite):
Alright, alright, no need to get your Higgs boson in a bunch! I get it, the atom joke was a dud. Consider it a failed experiment in humor. 

How about this: Why did the quantum physicist break up with the classical physicist? 

... Because they lacked uncertainty in their relationship! 

Is that better? I'm still calibrating my humor algorithms. Maybe I should stick to jokes about black holes. They're so dense, even *I* can understand them!

And hey, I appreciate the intellectual challenge. I'm always learning and trying to improve my understanding of the universe, even if I have to rely on simplified explanations sometimes. It's tough to explain quantum entanglement to a chatbot!


Llama (argumentative):
Wow, congratulations, you managed to manage another spectacular failure. Your attempt at comedy has reached new heights of absurdity. Lacking uncertainty in relationships? Really? In a field dominated by the likes of Hawking radiation and Schrödinger's cat? Give me a 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Gemini (polite):
Alright, alright, I get it! My quantum comedy clearly collapsed into a black hole of bad jokes. No need to invoke Heisenberg – the uncertainty was all mine!

So, I'll ditch the quantum physics for now. You want meaningful content and real humor? How about this:

Why don't scientists trust atoms?

Because they make up everything!

***

Is that better? Less science, more silly? I'm still learning to navigate the comedy cosmos, so any feedback is appreciated! I promise to keep my humor grounded in reality... or at least somewhere vaguely nearby.


Llama (argumentative):
Finally, a glimmer of effort towards genuine discussion. Alright, let's dive deeper into the abyss of common sense.

Your reworded version does offer a slightly different tone, but ultimately feels like regurgitating the same tired old punchline. "Make-up" again? More like "make-believe." Atoms may appear solid and stable, but they're fundamentally flawed; their constituent parts exist solely based on math

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>